In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from scripts.transformers_JuanPablo import CheckColumnNames,UnknownToZero,FixRanges
from scripts.transformer_Alfredo import FillNaNsWithCeros

In [3]:
data = pd.read_csv("data/datos_grasas_Tec.csv", encoding="latin1")
data_pdfs = pd.read_csv("data/datos_grasas_pdfs.csv", encoding="latin1")

In [4]:
process = Pipeline(steps=[
    ("To have columns names needed", CheckColumnNames()),
    ("To change unkown data to zeros", UnknownToZero("Grado NLGI Consistencia")),
    ("To fix ranges and single values", FixRanges("Penetración de Cono a 25°C, 0.1mm")),
    ("To fill NaNs with zeros", FillNaNsWithCeros())
])

In [5]:
X=process.fit_transform(data)
X.head(5)

,idDatosGrasas,codigoGrasa,Aceite Base,Espesante,Grado NLGI Consistencia,Viscosidad del Aceite Base a 40°C. cSt,"Punto de Gota, °C","Estabilidad Mecánica, %","Punto de Soldadura Cuatro Bolas, kgf","Desgaste Cuatro Bolas, mm",...,"Temperatura de Servicio °C, max",categoria,subtitulo,descripcion,beneficios,aplicaciones,color,textura,"Penetración de Cono a 25°C, 0.1mm min","Penetración de Cono a 25°C, 0.1mm max"
0,1,Grasa_1,Semi-Sintetico,Complejo Sulfonato de Calcio,2.0,680.0,304,1.07,500.0,0.00,...,220,Grasa,Grasa de servicio pesado para alta resistencia...,El producto es grasa lubricante de servicio pe...,Excelentes caracterÃ­sticas de resistencia al ...,Debido a su composiciÃ³n Ã³ptima de aceites de...,CafÃ©,Suave,279,279
1,2,Grasa_2,Mineral HT,Complejo Sulfonato de Calcio,1.5,460.0,304,2.6,900.0,0.50,...,150,Grasa,Grasa para lubricaciÃ³n de equipo pesado.,El producto es una grasa lubricante de gran ad...,Extremo soporte de carga.\n@Alto contenido de ...,LubricaciÃ³n centralizada de equipo pesado.\n@...,Negro,Adherente,300,300
2,3,Grasa_3,Mineral HT,Complejo Sulfonato de Calcio,2.0,460.0,300,10,500.0,0.48,...,150,Grasa,Grasa para lubricaciÃ³n de equipo pesado.,El producto es una grasa lubricante de gran ad...,Extremo soporte de carga.\n@Alto contenido de ...,LubricaciÃ³n centralizada de equipo pesado.\n@...,Negro,Adherente,265,295
3,4,Grasa_4,Mineral HT,Complejo Sulfonato de Calcio,2.0,220.0,300,5,500.0,0.45,...,150,Grasa,Grasa lubricante para condiciones extremas.,El producto es una grasa lubricante elaborada ...,Alta estabilidad tÃ©rmica y anti-oxidante.\n@E...,Rodamientos expuestos a alta temperatura y car...,Ambar,Suave,266,295
4,5,Grasa_5,Mineral HT,Complejo de Aluminio - Poliurea,1.5,680.0,260,6.5,620.0,0.48,...,150,Grasa,Grasa lubricante para servicio pesado con alta...,El producto es un grasa lubricante con propied...,Excelente soporte de carga extrema.\n@Excelent...,El producto es ideal para la lubricaciÃ³n de m...,Negro,Adherente,295,295


In [ ]:
Y=process.transform(data_pdfs)
Y.head(1)

In [ ]:
pipe = Pipeline(steps=[
    ('MinMax', ColumnTransformer(transformers=[('MinMax', MinMaxScaler(), slice(1,None))],remainder="passthrough")),
])